In [ ]:
# Basic imports
import numpy as np
import pandas as pd

from dataframe.csv_utils import (
    load_data_from_csv,
    get_labels_from_result,
    get_features_from_result,
)
from labels import print_label_count

# sklearn imports
from sklearn.decomposition import PCA  # Principal Component Analysis
from sklearn.manifold import TSNE  # T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans  # K-Means Clustering
from sklearn.preprocessing import StandardScaler  # used for 'Feature Scaling'

# plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

CLEANED_BETA_SUBJECT = [
    "../CleandDataV1/2017",
    "../CleandDataV1/2018",
    "../CleandDataV1/2020",
    "../CleandDataV1/2024",
    "../CleandDataV1/2025",
]

SORTED_BLOCK_NAMES = [
    "a_hvha",
    "a_hvla",
    "a_nvha",
    "a_nvla",
    "b_hvha",
    "b_hvla",
    "b_nvha",
    "b_nvla",
    "medi",
    "wandering",
]

all_blocks = []
for b in SORTED_BLOCK_NAMES:
    all_blocks.extend([b] * 13)

In [ ]:
"""
    load features from csv
"""

dir_name = "eeg_features"
result = load_data_from_csv(dir_name)

# Drop not cleaned beta subjects
subject = "20 subjects"
# mask = result["Subject"].isin([f"../CleandDataV1/{subject}"])
# result = result[mask]

all_label_array, label_list = get_labels_from_result(result, 0.65)
all_feature_array, feature_names = get_features_from_result(
    result, ["Subject", "Valence", "Arousal", "Attention"], False
)
# all_feature_array = all_feature_array.drop(["index"], axis=1)
feature_names = all_feature_array.columns
print(all_feature_array.shape, len(feature_names), len(label_list))
print_label_count(label_list)

In [ ]:
# Filter features
# get_other feature_pattern = ".*(?<!GAMMA|BETA2|BETA1|ALPHA|THETA|DELTA)$"
feature = 'DELTA'
only_specific_feature = f".*{feature}$" #f"^{feature}.*(?<!BETA2)$"

title = f"Subject {subject} EEG mean spectral power {feature}"
exclde_beta2_feature_pattern = ".*(?<!BETA2)$"
eeg_features = all_feature_array.filter(regex=only_specific_feature)
eeg_features.head()

In [ ]:
# Initialize our scaler
scaler = StandardScaler()
# Scale each column in numer
normalized_eeg_features = pd.DataFrame(scaler.fit_transform(eeg_features))
normalized_eeg_features.head()

In [ ]:
# reference https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/
reduced_data = PCA(n_components=0.95).fit_transform(normalized_eeg_features)
normalized_eeg_features = pd.DataFrame(reduced_data)
normalized_eeg_features.head()

In [ ]:
# from sklearn.mixture import GaussianMixture
# import matplotlib.pyplot as plt

# gmm = GaussianMixture(n_components=2).fit(normalized_eeg_features)
# clusters = gmm.predict(normalized_eeg_features)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import time

print("Compute unstructured hierarchical clustering...")
st = time.time()
ward = AgglomerativeClustering(n_clusters=4, linkage="ward").fit(
    normalized_eeg_features
)
elapsed_time = time.time() - st
clusters = ward.labels_
print(f"Elapsed time: {elapsed_time:.2f}s")
print(f"Number of points: {clusters.size}")

In [ ]:
from dataframe.visualization import pca_2d

# Add the cluster vector to our DataFrame, X
normalized_eeg_features["Cluster"] = clusters
# normalized_eeg_features['Block'] = all_blocks

pca_2d(
    normalized_eeg_features,
    4,
    [
        "rgba(255, 128, 255, 0.8)",
        "rgba(255, 128, 2, 0.8)",
        "rgba(0, 255, 200, 0.8)",
        "rgba(0, 128, 200, 0.8)",
    ],
    title,
    False,
    mode="markers",
    textfont=dict(size=10),
)

In [ ]:
# This is needed so we can display plotly plots properly
init_notebook_mode(connected=True)

In [ ]:
from plotly.subplots import make_subplots


def get_label_str(label_list: list) -> list:
    label_map = ["nvla", "nvha", "hvla", "hvha"]
    return [label_map[v] for v in label_list]


# Define color sets of paintings
colors_map = {
    "a_hvha": "rgb(177, 127, 38)",
    "a_hvla": "rgb(205, 152, 36)",
    "a_nvha": "rgb(99, 79, 37)",
    "a_nvla": "rgb(124, 103, 37)",
    "b_hvha": "rgb(33, 75, 99)",
    "b_hvla": "rgb(79, 129, 102)",
    "b_nvha": "rgb(151, 179, 100)",
    "b_nvla": "rgb(175, 49, 35)",
    "medi": "rgb(18, 36, 37)",
    "wandering": "rgb(56, 75, 126)",
    "nvla": "rgb(175, 49, 35)",
    "nvha": "rgb(99, 79, 37)",
    "hvla": "rgb(205, 152, 36)",
    "hvha": "rgb(33, 75, 99)",
}

# Create subplots, using 'domain' type for pie charts
specs = [
    [{"type": "domain"}, {"type": "domain"}],
    [{"type": "domain"}, {"type": "domain"}],
]
fig = make_subplots(rows=2, cols=2, specs=specs)


label_df = pd.DataFrame(get_label_str(label_list))
# Rename the columns
label_df.columns = ["label"]
plotX = pd.concat([normalized_eeg_features, label_df], axis=1, join="inner")

num_cluster = 4
groupBy = "label"
# Define pie charts
for i in range(num_cluster):
    c = plotX[plotX["Cluster"] == i].groupby([groupBy]).agg({groupBy: "count"})
    labels = c.index
    values = c[groupBy].tolist()
    colors = [colors_map[l] for l in labels]
    fig.add_trace(
        go.Pie(
            labels=labels,
            values=values,
            title_text=f"Cluster {i}",
            textinfo="label+percent",
            marker_colors=colors,
        ),
        int(i / 2 + 1),
        int(i % 2 + 1),
    )

# Tune layout and hover info
fig.update(
    layout_title_text=title,
    layout_showlegend=False,
)

fig = go.Figure(fig)
fig.show()

In [ ]:
# Clustering
# kmeans = KMeans(init="k-means++", n_clusters=4, n_init=2)
# kmeans.fit(normalized_eeg_features)
# #Find which cluster each data-point belongs to
# clusters = kmeans.predict(normalized_eeg_features)


# from sklearn.cluster import DBSCAN
# from sklearn import metrics

# db = DBSCAN(eps=7, min_samples=10).fit(normalized_eeg_features)
# clusters = db.labels_

# # Number of clusters in labels, ignoring noise if present.
# n_clusters_ = len(set(clusters)) - (1 if -1 in clusters else 0)
# n_noise_ = list(clusters).count(-1)

# print("Estimated number of clusters: %d" % n_clusters_)
# print("Estimated number of noise points: %d" % n_noise_)
# clusters

In [ ]:
# visualization via t-SNE
# Set our perplexity
perplexity = 50
# T-SNE with one dimension
tsne_1d = TSNE(n_components=1, perplexity=perplexity)

# T-SNE with two dimensions
tsne_2d = TSNE(n_components=2, perplexity=perplexity)

# T-SNE with three dimensions
tsne_3d = TSNE(n_components=3, perplexity=perplexity)
# This DataFrame holds a single dimension,built by T-SNE
s_1d = pd.DataFrame(
    tsne_1d.fit_transform(normalized_eeg_features.drop(["Cluster"], axis=1))
)

# This DataFrame contains two dimensions, built by T-SNE
s_2d = pd.DataFrame(
    tsne_2d.fit_transform(normalized_eeg_features.drop(["Cluster"], axis=1))
)

# And this DataFrame contains three dimensions, built by T-SNE
s_3d = pd.DataFrame(
    tsne_3d.fit_transform(normalized_eeg_features.drop(["Cluster"], axis=1))
)

s_1d.columns = ["TC1_1d"]

# "TC1_2d" means: 'The first component of the components created for 2-D visualization, by T-SNE.'
# And "TC2_2d" means: 'The second component of the components created for 2-D visualization, by T-SNE.'
s_2d.columns = ["TC1_2d", "TC2_2d"]

s_3d.columns = ["TC1_3d", "TC2_3d", "TC3_3d"]
method = "t-SNE"